Used ideas from 'Classification of Male and Female Speech Using Perceptual Features' DOI:10.1109/ICCCNT.2017.8204065

In [1]:
import os
from os.path import join
from os import listdir
import pandas as pd
import numpy as np
import librosa

from itertools import product

import IPython.display as ipd
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


from scipy.stats import skew

from utils.main_utils import get_readers, get_chapters_info, collect_paths_with_meta
from utils.scoring import cross_val

In [2]:
from typing import List, Tuple, Dict

In [3]:
audio_path = 'data/dev-clean/'
SEED = 0

N_CHROMA = 88
N_FOURIER_COEF = 30
FRAME_LENGTH = 1024           # sampling rate is 22kHZ, so 1024/220000 is approx 50 ms (as in paper)
HOP_LENGTH = FRAME_LENGTH // 2
SEC_PER_WINDOW = 2

In [4]:
def calc_abs_fouire_coefs(novelty_curve: np.ndarray, t: np.ndarray) -> np.ndarray:
    split_indxs = [0]+[np.where(np.floor(t)==(i+1)*SEC_PER_WINDOW)[0][0] for i in range(int(t[-1]//SEC_PER_WINDOW))]

    res = []
    for i in range(len(split_indxs)-1):
        res.append(np.abs(np.fft.fft(novelty_curve[split_indxs[i]:split_indxs[i+1]])[:N_FOURIER_COEF]))
    
    return np.array(res)

In [5]:
def calc_fourier_coefs_features(novelty_curve: np.ndarray, sr: int) -> np.ndarray:
    t = librosa.frames_to_time(range(len(novelty_curve)), sr, hop_length=HOP_LENGTH, n_fft=FRAME_LENGTH)
    
    coefs_by_windows = calc_abs_fouire_coefs(novelty_curve, t)
    coefs_mean = coefs_by_windows.mean(0)
    coefs_std = coefs_by_windows.std(0)

    return [coefs_mean, coefs_std]

In [6]:
def get_features(sample_path: str) ->np.ndarray:
    y, sr = librosa.load(sample_path)
    
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=N_CHROMA, hop_length=HOP_LENGTH, n_fft=FRAME_LENGTH)
    zcr = librosa.feature.zero_crossing_rate(y, hop_length=HOP_LENGTH, frame_length=FRAME_LENGTH)
    rms = librosa.feature.rms(y, hop_length=HOP_LENGTH, frame_length=FRAME_LENGTH)
    novelty_curve = librosa.onset.onset_strength(y, sr=sr)
    skewness = skew(y)
    fourier_coefs_features = calc_fourier_coefs_features(novelty_curve, sr)
    
    features = [chroma_stft.mean(1)]
    for el in [zcr, rms, novelty_curve[None, :]]:
        features.append(el.mean(1))
        features.append(el.std(1))
    features.append(skewness)
    
    return np.hstack(features+fourier_coefs_features)

In [7]:
def process_batch(samples_paths: List[str]) -> Dict[str, np.ndarray]:
    res = {}
    for sample_path in samples_paths:
        res[sample_path] = get_features(sample_path)
    return res

In [8]:
def generate_features(meta_paths: pd.DataFrame, features_names: List[str],
                      n_jobs: int, batch_size: int) -> pd.DataFrame:
    
    batch_number = int(np.ceil(len(meta_paths)/batch_size))
    print(f'batch number: {batch_number}')
    jobs = []
    for i in range(batch_number):
        jobs.append(delayed(process_batch)(meta_paths.path[i*batch_size:(i+1)*batch_size].values))
    features = Parallel(n_jobs=n_jobs, verbose=10)(jobs)

    features_dict = {}
    for el in features:
        for k,v in el.items():
            features_dict[k] = v
    features_df = pd.DataFrame(features_dict).T
    features_df.columns = features_names
    features_df.index.name = 'path'
    
    return features_df

In [9]:
readers = get_readers('data/speakers.tsv', audio_path)
chapter_info = get_chapters_info('./data/CHAPTERS.txt')
meta_paths = collect_paths_with_meta(audio_path, readers)
durations = meta_paths.path.apply(lambda x: librosa.get_duration(filename=x))
meta_paths = meta_paths[durations>=SEC_PER_WINDOW]

In [10]:
features_names = [f'chroma_stft_{i}_{aggr}' for i, aggr in product(range(N_CHROMA), ['mean'])]
features_names += [f'{method}_{aggr}' for method, aggr in product(['zcr', 'rms', 'spectral_flux'],
                                                                 ['mean', 'std'])]
features_names.append('skewness_mean')
features_names += [f'fourier_{i}_{aggr}' for i, aggr in product(range(N_FOURIER_COEF), ['mean', 'std'])]
len(features_names)

155

In [11]:
features = generate_features(meta_paths, features_names, n_jobs=12, batch_size=100)
features.head()

batch number: 47


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   22.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:   31.4s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   52.5s
[Parallel(n_jobs=12)]: Done  29 out of  47 | elapsed:  1.3min remaining:   48.3s
[Parallel(n_jobs=12)]: Done  34 out of  47 | elapsed:  1.4min remaining:   31.6s
[Parallel(n_jobs=12)]: Done  39 out of  47 | elapsed:  1.7min remaining:   21.0s
[Parallel(n_jobs=12)]: Done  44 out of  47 | elapsed:  1.8min remaining:    7.3s
[Parallel(n_jobs=12)]: Done  47 out of  47 | elapsed:  1.8min finished


,chroma_stft_0_mean,chroma_stft_1_mean,chroma_stft_2_mean,chroma_stft_3_mean,chroma_stft_4_mean,chroma_stft_5_mean,chroma_stft_6_mean,chroma_stft_7_mean,chroma_stft_8_mean,chroma_stft_9_mean,...,fourier_25_mean,fourier_25_std,fourier_26_mean,fourier_26_std,fourier_27_mean,fourier_27_std,fourier_28_mean,fourier_28_std,fourier_29_mean,fourier_29_std
path,,,,,,,,,,,,,,,,,,,,,
data/dev-clean/1919/142785/1919_142785_000005_000002.wav,0.257667,0.260941,0.260627,0.260974,0.264403,0.268702,0.265345,0.259730,0.259609,0.261992,...,7.247560,6.834326,11.152557,7.371055,2.139417,6.980699,6.690299,8.143005,5.156007,6.926347
data/dev-clean/1919/142785/1919_142785_000118_000001.wav,0.351155,0.339509,0.331138,0.324826,0.317682,0.311728,0.303800,0.296491,0.297595,0.300528,...,9.900573,6.351802,13.242432,8.743847,13.568657,9.211354,7.838845,2.233069,6.773000,7.983278
data/dev-clean/1919/142785/1919_142785_000035_000001.wav,0.216299,0.212288,0.216035,0.218830,0.221038,0.224242,0.225550,0.229322,0.233747,0.234828,...,4.503046,4.584075,5.707855,8.854382,4.414703,2.194819,3.986808,2.231112,11.456552,3.195519
data/dev-clean/1919/142785/1919_142785_000064_000003.wav,0.306730,0.291617,0.290036,0.291296,0.291558,0.287382,0.281830,0.282077,0.286006,0.288771,...,7.970376,12.983988,7.974744,7.372138,9.622538,8.023809,8.653289,2.348849,5.060957,6.579962
data/dev-clean/1919/142785/1919_142785_000059_000000.wav,0.264273,0.270635,0.257493,0.256361,0.259760,0.255054,0.249914,0.238382,0.228974,0.231650,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
dataset = meta_paths.join(features, on='path')
dataset.gender = dataset.gender.apply(lambda x: int(x=='F'))
dataset.head()

,reader,gender,path,chroma_stft_0_mean,chroma_stft_1_mean,chroma_stft_2_mean,chroma_stft_3_mean,chroma_stft_4_mean,chroma_stft_5_mean,chroma_stft_6_mean,...,fourier_25_mean,fourier_25_std,fourier_26_mean,fourier_26_std,fourier_27_mean,fourier_27_std,fourier_28_mean,fourier_28_std,fourier_29_mean,fourier_29_std
0,1919,1,data/dev-clean/1919/142785/1919_142785_000005_...,0.257667,0.260941,0.260627,0.260974,0.264403,0.268702,0.265345,...,7.247560,6.834326,11.152557,7.371055,2.139417,6.980699,6.690299,8.143005,5.156007,6.926347
1,1919,1,data/dev-clean/1919/142785/1919_142785_000118_...,0.351155,0.339509,0.331138,0.324826,0.317682,0.311728,0.303800,...,9.900573,6.351802,13.242432,8.743847,13.568657,9.211354,7.838845,2.233069,6.773000,7.983278
2,1919,1,data/dev-clean/1919/142785/1919_142785_000035_...,0.216299,0.212288,0.216035,0.218830,0.221038,0.224242,0.225550,...,4.503046,4.584075,5.707855,8.854382,4.414703,2.194819,3.986808,2.231112,11.456552,3.195519
3,1919,1,data/dev-clean/1919/142785/1919_142785_000064_...,0.306730,0.291617,0.290036,0.291296,0.291558,0.287382,0.281830,...,7.970376,12.983988,7.974744,7.372138,9.622538,8.023809,8.653289,2.348849,5.060957,6.579962
5,1919,1,data/dev-clean/1919/142785/1919_142785_000059_...,0.264273,0.270635,0.257493,0.256361,0.259760,0.255054,0.249914,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
dataset.shape

(4664, 158)

In [14]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
pca = PCA(n_components=50, random_state=SEED)

In [15]:
first_feature_indx = dataset.columns.to_list().index('path') + 1

In [16]:
clf = SVC(gamma='scale', kernel='poly', degree=2, random_state=SEED)
_, mean_scores = cross_val(clf, readers, dataset, kf=skf, decomposer=pca, ffi=first_feature_indx)
print(mean_scores)

{'accuracy': 0.52, 'f1': 0.6, 'matthews': 0.07}


In [17]:
clf = SVC(gamma='scale', kernel='poly', degree=2, random_state=SEED)
_, mean_scores = cross_val(clf, readers, dataset, kf=skf, decomposer=None, ffi=first_feature_indx)
print(mean_scores)

{'accuracy': 0.6, 'f1': 0.63, 'matthews': 0.22}


In [18]:
clf = SVC(gamma='scale', kernel='rbf', random_state=SEED)
_, mean_scores = cross_val(clf, readers, dataset, kf=skf, decomposer=None, ffi=first_feature_indx)
print(mean_scores)

{'accuracy': 0.6, 'f1': 0.63, 'matthews': 0.23}


In [19]:
clf = KNeighborsClassifier(n_neighbors=3, p=2)
_, mean_scores = cross_val(clf, readers, dataset, kf=skf, decomposer=pca, ffi=first_feature_indx)
print(mean_scores)

{'accuracy': 0.57, 'f1': 0.6, 'matthews': 0.16}


In [20]:
clf = KNeighborsClassifier(n_neighbors=3, p=2)
_, mean_scores = cross_val(clf, readers, dataset, kf=skf, decomposer=None, ffi=first_feature_indx)
print(mean_scores)

{'accuracy': 0.56, 'f1': 0.6, 'matthews': 0.15}


In [21]:
rf = RandomForestClassifier(random_state=SEED)
_, mean_scores = cross_val(rf, readers, dataset, kf=skf, decomposer=pca, ffi=first_feature_indx)
print(mean_scores)

{'accuracy': 0.6, 'f1': 0.61, 'matthews': 0.23}


In [22]:
rf = RandomForestClassifier(random_state=SEED)
_, mean_scores = cross_val(rf, readers, dataset, kf=skf, decomposer=None, ffi=first_feature_indx)
print(mean_scores)

{'accuracy': 0.81, 'f1': 0.82, 'matthews': 0.64}


In [23]:
fi_args = np.argsort(rf.feature_importances_)[::-1]
[features_names[i] for i in fi_args]

['chroma_stft_68_mean',
 'chroma_stft_66_mean',
 'chroma_stft_69_mean',
 'chroma_stft_71_mean',
 'chroma_stft_67_mean',
 'chroma_stft_70_mean',
 'chroma_stft_74_mean',
 'chroma_stft_64_mean',
 'skewness_mean',
 'chroma_stft_72_mean',
 'chroma_stft_62_mean',
 'chroma_stft_65_mean',
 'chroma_stft_63_mean',
 'chroma_stft_75_mean',
 'zcr_std',
 'chroma_stft_60_mean',
 'chroma_stft_78_mean',
 'chroma_stft_83_mean',
 'chroma_stft_73_mean',
 'chroma_stft_85_mean',
 'rms_mean',
 'chroma_stft_76_mean',
 'chroma_stft_14_mean',
 'chroma_stft_77_mean',
 'chroma_stft_81_mean',
 'zcr_mean',
 'chroma_stft_80_mean',
 'chroma_stft_18_mean',
 'chroma_stft_45_mean',
 'chroma_stft_61_mean',
 'chroma_stft_17_mean',
 'chroma_stft_16_mean',
 'chroma_stft_42_mean',
 'spectral_flux_std',
 'chroma_stft_22_mean',
 'chroma_stft_54_mean',
 'chroma_stft_21_mean',
 'chroma_stft_79_mean',
 'chroma_stft_87_mean',
 'spectral_flux_mean',
 'chroma_stft_57_mean',
 'chroma_stft_19_mean',
 'fourier_0_mean',
 'fourier_0_std'